In [1]:
pip install pyradiomics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.3/188.3 kB 4.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.1/500.1 kB 31.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tarfile
file = tarfile.open('../input/brats-2021-task1/BraTS2021_Training_Data.tar')

file.extractall('./brain_images')
file.close()

In [3]:
import numpy as np
import re
import nibabel as nib
import glob
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() 
import tensorflow as tf
import SimpleITK as sitk
from radiomics import featureextractor
from scipy import ndimage
import six
from skimage.exposure import histogram
from skimage.util import img_as_ubyte
from skimage import exposure, util
from tensorflow.keras.utils import to_categorical
import os


In [4]:
def centre_of_tumor(input_image,input_mask):
  extractor = featureextractor.RadiomicsFeatureExtractor()
  extractor.disableAllFeatures()
  result = extractor.execute(input_image,input_mask)
  print(result)
  centre_of_mass = []
  for key, value in six.iteritems(result):
    if key == 'diagnostics_Mask-original_CenterOfMassIndex':
      centre_of_mass.append(value[0])
      centre_of_mass.append(value[1])
      centre_of_mass.append(value[2]) 
  return centre_of_mass

In [5]:
def crop_images(img,centre_of_mass,optimal_roi):
  
  x_roi = round(optimal_roi[0]/2)
  y_roi = round(optimal_roi[1]/2)
  z_roi = round(optimal_roi[2]/2)
  x_centre = round(centre_of_mass[0])
  y_centre = round(centre_of_mass[1])
  z_centre= round(centre_of_mass[2])

  sizeX, sizeY, sizeZ = img.GetSize() # Get image dimensions using GetSize()

  if x_centre - x_roi < 0:
    start_x = 0 
    end_x = optimal_roi[0]
  elif x_centre + x_roi >= sizeX:
    end_x = sizeX - 1
    start_x = end_x - optimal_roi[0]
  else:
    start_x = x_centre - x_roi
    end_x =  x_centre + x_roi

  if y_centre - y_roi < 0:
    start_y = 0 
    end_y = optimal_roi[1]
  elif y_centre + y_roi >= sizeY:
    end_y = sizeY - 1
    start_y = end_y - optimal_roi[1]
  else:
    start_y = y_centre - y_roi
    end_y =  y_centre + y_roi

  if z_centre - z_roi < 0:
    start_z = 0 
    end_z = optimal_roi[2]
  elif z_centre + z_roi >= sizeZ:
    end_z = sizeZ - 1
    start_z = end_z - optimal_roi[2]
  else:
    start_z = z_centre - z_roi
    end_z =  z_centre + z_roi

  cropped_img = img[start_x:end_x,
                    start_y:end_y,
                    start_z:end_z] 
  return cropped_img


In [6]:
optimal_roi = [128,128,128]
image_size = [240,240,155]

In [7]:
t1_list = sorted(glob.glob('./brain_images/*/*t1.nii.gz'))
t2_list = sorted(glob.glob('./brain_images/*/*t2.nii.gz'))
t1ce_list = sorted(glob.glob('./brain_images/*/*t1ce.nii.gz'))
flair_list = sorted(glob.glob('./brain_images/*/*flair.nii.gz'))
mask_list = sorted(glob.glob('./brain_images/*/*seg.nii.gz'))
pattern = re.compile('./brain_images/.*_(\w*)\.nii\.gz')

In [8]:
data_paths = [{
    pattern.findall(item)[0]:item for item in items
}
for items in list(zip(t1_list, t2_list, t1ce_list, flair_list, mask_list))]

print('number of training examples' ,len(data_paths))
print(data_paths[0])

number of training examples 1251
{'t1': './brain_images/BraTS2021_00000/BraTS2021_00000_t1.nii.gz', 't2': './brain_images/BraTS2021_00000/BraTS2021_00000_t2.nii.gz', 't1ce': './brain_images/BraTS2021_00000/BraTS2021_00000_t1ce.nii.gz', 'flair': './brain_images/BraTS2021_00000/BraTS2021_00000_flair.nii.gz', 'seg': './brain_images/BraTS2021_00000/BraTS2021_00000_seg.nii.gz'}


In [9]:
def rescale_image(npimage):
  image= sitk.GetImageFromArray(npimage)
  filter = sitk.RescaleIntensityImageFilter()
  filter.SetOutputMaximum(255)
  filter.SetOutputMinimum(0)
  rescaled_img = filter.Execute(image)
  return rescaled_img

In [10]:
def rescale_nene(input_image):
    # Define the rescale factor
    rescale_factor = 1

# Define the output size based on the input image size and rescale factor
    output_size = [int(round(sz*rescale_factor)) for sz in input_image.GetSize()]

# Create the resampling filter
    resampler = sitk.ResampleImageFilter()

# Set the output pixel type to match the input image pixel type
    resampler.SetOutputPixelType(input_image.GetPixelID())

# Set the output size and spacing based on the rescale factor
    resampler.SetSize(output_size)
    resampler.SetOutputSpacing([sz*rescale_factor for sz in input_image.GetSpacing()])

# Set the interpolation method to linear
    resampler.SetInterpolator(sitk.sitkLinear)

# Resample the input image using the defined rescale factor
    output_image = resampler.Execute(input_image)
    
    return output_image

In [11]:
def normalize_image(image):
    mean = np.mean(image)
    std = np.std(image)
    normalized_image = (image - mean) / std
    return normalized_image

In [12]:
def n4_bias_correction(image):
  shrinkFactor = 1
  #image = sitk.ReadImage(sys.argv[1])
  image = sitk.Shrink(image, [shrinkFactor] * image.GetDimension())
  image = sitk.Cast(image,sitk.sitkFloat32)
  corrector = sitk.N4BiasFieldCorrectionImageFilter()
  corrector.SetNumberOfControlPoints([4,4,4])
  corrector.SetConvergenceThreshold(0.0001)
  corrector.SetWienerFilterNoise=0.01

  output_corrected = corrector.Execute(image)
  return output_corrected

In [13]:
    def read_img_sitk(img_path):
        image_data = sitk.ReadImage(img_path)
        return image_data 

In [ ]:
for img in range(len(t2_list)):   #Using t2_list as all lists are of same size
    print("Now preparing image and masks number: ", img)
    
        #cropping
    mask = read_img_sitk(data_paths[30]['seg'])
    imaget1 = read_img_sitk(data_paths[img]['t1'])
    centre = centre_of_tumor(imaget1,mask)
    temp_image_t1_c = crop_images(imaget1,centre,optimal_roi)
    
    imaget1ce = read_img_sitk(data_paths[img]['t1ce'])
    temp_image_t1ce_c = crop_images(imaget1ce,centre,optimal_roi)
    
    imaget2 = read_img_sitk(data_paths[img]['t2'])
    temp_image_t2_c = crop_images(imaget2,centre,optimal_roi)
    
    imagetflair = read_img_sitk(data_paths[img]['flair'])
    temp_image_flair_c = crop_images(imagetflair,centre,optimal_roi)
    
    temp_mask_c = crop_images(mask,centre,optimal_roi)
    
    print(type(temp_image_t1_c))
    print(type(temp_image_t1ce_c))
    print(type(temp_image_t2_c))
    print(type(temp_image_flair_c))
    print(type(temp_mask_c))
    
    
         #rescaling
    r_temp_image_t1 = rescale_nene(temp_image_t1_c)
    r_temp_image_t1ce = rescale_nene(temp_image_t1ce_c)
    r_temp_image_t2 = rescale_nene(temp_image_t2_c)
    r_temp_image_flair = rescale_nene(temp_image_flair_c)
    r_temp_mask = rescale_nene(temp_mask_c)
    
    print("Rescaling done...")
    
    print(type(r_temp_image_t1))
    print(type(r_temp_image_t1ce))
    print(type(r_temp_image_t2))
    print(type(r_temp_image_flair))
    print(type(r_temp_mask))
    
    np_temp_image_t1_c = sitk.GetArrayFromImage(r_temp_image_t1)
    np_temp_image_t1ce_c = sitk.GetArrayFromImage(r_temp_image_t1ce)
    np_temp_image_t2_c = sitk.GetArrayFromImage(r_temp_image_t2)
    np_temp_image_flair_c = sitk.GetArrayFromImage(r_temp_image_flair)
    np_temp_mask_c = sitk.GetArrayFromImage(r_temp_mask)
    
    print(np_temp_image_t1_c.shape)
    print(np_temp_image_t1ce_c.shape)
    print(np_temp_image_t2_c.shape)
    print(np_temp_image_flair_c.shape)
    print(np_temp_mask_c.shape)
    
    print("Shape changed to numpy array")
    
    n_temp_image_t1 = normalize_image(np_temp_image_t1_c)
    n_temp_image_t1ce = normalize_image(np_temp_image_t1ce_c)
    n_temp_image_t2 = normalize_image(np_temp_image_t2_c)
    n_temp_image_flair = normalize_image(np_temp_image_flair_c)
    n_temp_mask = normalize_image(np_temp_mask_c)
    print("Normalizing done")
    
        #image bias correction
    
    i_temp_image_t1 = sitk.GetImageFromArray(n_temp_image_t1)
    i_temp_image_t1ce = sitk.GetImageFromArray(n_temp_image_t1ce)
    i_temp_image_t2 = sitk.GetImageFromArray(n_temp_image_t2)
    i_temp_image_flair = sitk.GetImageFromArray(n_temp_image_flair)
    i_temp_mask = sitk.GetImageFromArray(n_temp_mask)
    
    n4_temp_image_t1 = n4_bias_correction(i_temp_image_t1)
    n4_temp_image_t1ce = n4_bias_correction(i_temp_image_t1ce)
    n4_temp_image_t2 = n4_bias_correction(i_temp_image_t2)
    n4_temp_image_flair = n4_bias_correction(i_temp_image_flair)
    n4_temp_mask = n4_bias_correction(i_temp_mask)
    print("N4 Biasing done")
    
    
    npl_temp_image_t1_n = sitk.GetArrayFromImage(n4_temp_image_t1)
    npl_temp_image_t1ce_n = sitk.GetArrayFromImage(n4_temp_image_t1ce)
    npl_temp_image_t2_n = sitk.GetArrayFromImage(n4_temp_image_t2)
    npl_temp_image_flair_n = sitk.GetArrayFromImage(n4_temp_image_flair)
    npl_temp_mask_n = sitk.GetArrayFromImage(n4_temp_mask)
    
    print(npl_temp_image_t1_n.shape)
    print(npl_temp_image_t1ce_n.shape)
    print(npl_temp_image_t2_n.shape)
    print(npl_temp_image_flair_n.shape)
    print(npl_temp_mask_n.shape)
    
    #combining
    
    temp_combined_images = np.stack([npl_temp_image_t1_n, npl_temp_image_t1ce_n, npl_temp_image_t2_n, npl_temp_image_flair_n], axis=-1)
    
    val, counts = np.unique(npl_temp_mask_n, return_counts=True)
    
    if not os.path.exists('images'):
        os.makedirs('images')
        
    if not os.path.exists('masks'):
        os.makedirs('masks')
    
    if (1 - (counts[0]/counts.sum())) > 0.01:  #At least 1% useful volume with labels that are not 0
        print("Save Me")
        if len(n_temp_mask) > 4:
            n_temp_mask = n_temp_mask[:4]
        n_temp_mask= to_categorical(n_temp_mask, num_classes=4)
        np.save('/kaggle/working/images/image_'+str(img)+'.npy', temp_combined_images)
        np.save('/kaggle/working/masks/mask_'+str(img)+'.npy', n_temp_mask)
        
    else:
        print("I am useless...")
        
    print("Onto next image...")   

    

Now preparing image and masks number:  0
OrderedDict([('diagnostics_Versions_PyRadiomics', 'v3.0.1'), ('diagnostics_Versions_Numpy', '1.21.6'), ('diagnostics_Versions_SimpleITK', '2.2.1'), ('diagnostics_Versions_PyWavelet', '1.3.0'), ('diagnostics_Versions_Python', '3.7.12'), ('diagnostics_Configuration_Settings', {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True}), ('diagnostics_Configuration_EnabledImageTypes', {'Original': {}}), ('diagnostics_Image-original_Hash', '7089153d1311a61672379293141276015fbae37e'), ('diagnostics_Image-original_Dimensionality', '3D'), ('diagnostics_Image-original_Spacing', (1.0, 1.0, 1.0)), ('diagnostics_Image-original_Size', (240, 240, 155)), ('diagnostics_Image-original_Mean', 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


N4 Biasing done
(128, 128, 128)
(128, 128, 128)
(128, 128, 128)
(128, 128, 128)
(128, 128, 128)
I am useless...
Onto next image...
Now preparing image and masks number:  1
OrderedDict([('diagnostics_Versions_PyRadiomics', 'v3.0.1'), ('diagnostics_Versions_Numpy', '1.21.6'), ('diagnostics_Versions_SimpleITK', '2.2.1'), ('diagnostics_Versions_PyWavelet', '1.3.0'), ('diagnostics_Versions_Python', '3.7.12'), ('diagnostics_Configuration_Settings', {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True}), ('diagnostics_Configuration_EnabledImageTypes', {'Original': {}}), ('diagnostics_Image-original_Hash', '0881935c68d6cad3f5ed0b5597467a52a47a5017'), ('diagnostics_Image-original_Dimensionality', '3D'), ('diagnostics_Im